# World Bank's World Development Indicators vs. Environment Analysis - Analysis Notebook

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

In [6]:
%run ./help_funcs.ipynb

## Load the datasets

Climate Change/Pollution Indicators

| Dataset | Series Code | Series Description/Name |
| --- | --- | --- |
|CO2| EN.ATM.CO2E.KT | CO2 emissions (kt)|
|Other Gas | EN.ATM.GHGT.KT.CE | Total greenhouse gas emissions (kt of CO2 equivalent) |
|Other Gas | EN.ATM.METH.KT.CE | Methane emissions (kt of CO2 equivalent)|
|Other Gas | EN.ATM.NOXE.KT.CE | Nitrous oxide emissions (thousand metric tons of CO2 equivalent)|


World Development Indicators

| Dataset | Series Code | Series Description/Name |
| --- | --- | --- |
| GDP | NY.GDP.MKTP.KD.ZG | GDP growth (annual %) |
| GDP | NY.GDP.PCAP.KD.ZG | GDP per capita growth (annual %) |
| Land Use | AG.LND.FRST.K2 | Forest area (sq. km) |
| Land Use | AG.LND.FRST.ZS | Forest area (% of land area) |
| Urbanization | SP.POP.GROW | Population growth (annual %) |
| Urbanization | SP.RUR.TOTL.ZG | Rural population growth (annual %) |
| Urbanization | SP.URB.GROW | Urban population growth (annual %) |

In [7]:
countries_list = ['China', 'United States', 'India', 'Russian Federation', 'Japan']

In [8]:
co2_descdf, co2_df = world_bank_file_to_df('../../01_DataSources/01_Raw/World_Bank_CO2_World_Development_Indicators/World_Bank_CO2_World_Development_Indicators_Data.csv')
co2_df = co2_df[co2_df['Country'].isin(countries_list)]
co2_df = co2_df[~co2_df['Year'].isin([2020, 2021])]
co2_df = co2_df[['Country', 'Year', 'EN.ATM.CO2E.KT']]
print('CO2 df shape', co2_df.shape)

othergas_descdf, othergas_df = world_bank_file_to_df('../../01_DataSources/01_Raw/World_Bank_Othergas_World_Development_Indicators/World_Bank_Othergas_World_Development_Indicators_Data.csv')
othergas_df = othergas_df[othergas_df['Country'].isin(countries_list)]
othergas_df = othergas_df[~othergas_df['Year'].isin([2020, 2021])]
othergas_df = othergas_df[['Country', 'Year', 'EN.ATM.GHGT.KT.CE', 'EN.ATM.METH.KT.CE', 'EN.ATM.NOXE.KT.CE']]
print('Other Gas df shape', othergas_df.shape)

gdp_descdf, gdp_df = world_bank_file_to_df('../../01_DataSources/01_Raw/World_Bank_GDP_World_Development_Indicators/World_Bank_GDP_World_Development_Indicators_Data.csv')
gdp_df = gdp_df[gdp_df['Country'].isin(countries_list)]
gdp_df = gdp_df[~gdp_df['Year'].isin([2020, 2021])]
gdp_df = gdp_df[['Country', 'Year', 'NY.GDP.MKTP.KD.ZG', 'NY.GDP.PCAP.KD.ZG']]
print('GDP df shape', gdp_df.shape)

landuse_descdf, landuse_df = world_bank_file_to_df('../../01_DataSources/01_Raw/World_Bank_LandUse_World_Development_Indicators/World_Bank_LandUse_World_Development_Indicators_Data.csv')
landuse_df = landuse_df[landuse_df['Country'].isin(countries_list)]
landuse_df = landuse_df[~landuse_df['Year'].isin([2020, 2021])]
landuse_df = landuse_df[['Country', 'Year', 'AG.LND.FRST.K2', 'AG.LND.FRST.ZS']]
print('Land Use df shape', landuse_df.shape)

urb_descdf, urb_df = world_bank_file_to_df('../../01_DataSources/01_Raw/World_Bank_Urbanization_World_Development_Indicators/World_Bank_Urbanization_World_Development_Indicators_Data.csv')
urb_df = urb_df[urb_df['Country'].isin(countries_list)]
urb_df = urb_df[~urb_df['Year'].isin([2020, 2021])]
urb_df = urb_df[['Country', 'Year', 'SP.POP.GROW', 'SP.RUR.TOTL.ZG', 'SP.URB.GROW']]
print('Urbanization df shape', urb_df.shape)

CO2 df shape (65, 3)
Other Gas df shape (65, 5)
GDP df shape (65, 4)
Land Use df shape (65, 4)
Urbanization df shape (65, 5)


In [9]:
resdf = co2_df.merge(othergas_df, how='left', on=['Country', 'Year'])\
        .merge(gdp_df, how='left', on=['Country', 'Year'])\
        .merge(landuse_df, how='left', on=['Country', 'Year'])\
        .merge(urb_df, how='left', on=['Country', 'Year'])

print('Result df shape =', resdf.shape)

Result df shape = (65, 13)
